In [1]:
import polars as pl
import importlib

import Utils.config as configs
import Utils.grid_sampling as grid_samp
import Utils.grid_generation as grid_gen
import Utils.force_frame as force_frame
import Utils.path_optimisation as path_opt
import Utils.visualisation as vis
import Utils.book_keeping as bk

importlib.reload(configs)
importlib.reload(grid_samp)
importlib.reload(grid_gen)
importlib.reload(force_frame)
importlib.reload(path_opt)
importlib.reload(vis)
importlib.reload(bk)

<module 'Utils.book_keeping' from 'c:\\Users\\johan\\OneDrive\\Documents\\Python\\Python\\SilverStone\\Utils\\book_keeping.py'>

In [ ]:
# powers = [100, 200, 300, 450, 600, 750, 900, 1050, 1200, 1350,1500,1800,2100,2400,2700,3000,3300]
# n_lanes = [1,11]
# control_trusts = [0.1,0.3,0.4, 0.5,0.75,1.0]
# limit_trusts = [0.75,0.8,0.85,0.9,0.95,1.0]

for n_lanes in n_lanes:
    for power in powers:
        config = configs.config.modelV1(power,n_lanes=n_lanes)
        grid = grid_gen.Grid(config, configs.folder_config.TRACK_FOLDER + "\\TrackData.csv")
        df = grid.data_generation(verbose=False)
        for limit_trust in limit_trusts:
            config.pc.limit_trust = limit_trust
            costs_df = force_frame.Forces.add_Forces_and_constraints(config, df)
            for control_trust in control_trusts:
                print(f"n_lanes:{n_lanes} power:{power} limit_trust:{limit_trust} control_trust:{control_trust}")
                config.pc.control_trust = control_trust
                traj,V = path_opt.Optimisation.run_stack(config, costs_df, save_run=True,  show_plots=False, verbose=False)            

original length: 1178 resampled lenght 5887 df_reduced_len 404
n_lanes:1 power:100 limit_trust:0.75 control_trust:0.1
tot_cost:336.1938777524172 tot_cost_robust=335.8680348672176 max_speed=19.0

n_lanes:1 power:100 limit_trust:0.75 control_trust:0.3
tot_cost:335.4361188411701 tot_cost_robust=335.19209663022247 max_speed=19.0

n_lanes:1 power:100 limit_trust:0.75 control_trust:0.4
tot_cost:334.8275169487835 tot_cost_robust=334.37072737735934 max_speed=19.0

n_lanes:1 power:100 limit_trust:0.75 control_trust:0.5
tot_cost:334.20945522885285 tot_cost_robust=333.8485602749407 max_speed=19.0

n_lanes:1 power:100 limit_trust:0.75 control_trust:0.75
tot_cost:332.59706874131257 tot_cost_robust=332.2639663397579 max_speed=19.0

n_lanes:1 power:100 limit_trust:0.75 control_trust:1.0
tot_cost:329.7438898114756 tot_cost_robust=329.74388981147536 max_speed=19.0

n_lanes:1 power:100 limit_trust:0.8 control_trust:0.1
Skip
n_lanes:1 power:100 limit_trust:0.8 control_trust:0.3
tot_cost:330.1261568756307

In [3]:

powers = [100, 200, 300, 450, 600, 750, 900, 1050, 1200, 1350,1500,1800,2100,2400,2700,3000,3300]
n_lanes = [1,11]
control_trusts = [0.1,0.3,0.4, 0.5,0.75,1.0]
limit_trusts = [0.75,0.8,0.85,0.9,0.95,1.0]

for n_lanes in n_lanes:
    for power in powers:
        #Pass 1
        config = configs.config.modelV1(power,n_lanes=n_lanes)
        grid = grid_gen.Grid(config, configs.folder_config.TRACK_FOLDER + "\\TrackData.csv")
        df = grid.data_generation(verbose=False)
        costs_df = force_frame.Forces.add_Forces_and_constraints(config, df)            
        traj,V = path_opt.Optimisation.run_stack(config, costs_df, show_plots=False, verbose=False, save_run=False)
        df_with_trajectory = traj.select(["tr_x", "tr_y"]).with_columns(
                    [pl.col("tr_x").alias("x_lane0"), pl.col("tr_y").alias("y_lane0")])
        for limit_trust in limit_trusts:
            for control_trust in control_trusts:
                print(f"n_lanes:{n_lanes} power:{power} limit_trust:{limit_trust} control_trust:{control_trust}")
                #Pass 2
                config_pass2 = configs.config.modelV1(power,n_lanes=1)
                config_pass2.pc.control_trust = control_trust
                config_pass2.pc.limit_trust = limit_trust
                grid = grid_gen.Grid(config_pass2, configs.folder_config.TRACK_FOLDER + "\\TrackData.csv")
                df = grid.data_generation_from_existing_track(df_with_trajectory)
                costs_df = force_frame.Forces.add_Forces_and_constraints(config_pass2, df)
                traj_2,V_2 = path_opt.Optimisation.run_stack(config_pass2, costs_df, save_run=True, show_plots=False, verbose=False, suffix="TwoPass")
                


original length: 1178 resampled lenght 5887 df_reduced_len 404
tot_cost:284.1136017789677 tot_cost_robust=284.11360177896734 max_speed=22.166666666666668

n_lanes:1 power:100 limit_trust:0.75 control_trust:0.1
Skip
n_lanes:1 power:100 limit_trust:0.75 control_trust:0.3
Skip
n_lanes:1 power:100 limit_trust:0.75 control_trust:0.4
Skip
n_lanes:1 power:100 limit_trust:0.75 control_trust:0.5
Skip
n_lanes:1 power:100 limit_trust:0.75 control_trust:0.75
Skip
n_lanes:1 power:100 limit_trust:0.75 control_trust:1.0
Skip
n_lanes:1 power:100 limit_trust:0.8 control_trust:0.1
Skip
n_lanes:1 power:100 limit_trust:0.8 control_trust:0.3
Skip
n_lanes:1 power:100 limit_trust:0.8 control_trust:0.4
Skip
n_lanes:1 power:100 limit_trust:0.8 control_trust:0.5
Skip
n_lanes:1 power:100 limit_trust:0.8 control_trust:0.75
Skip
n_lanes:1 power:100 limit_trust:0.8 control_trust:1.0
Skip
n_lanes:1 power:100 limit_trust:0.85 control_trust:0.1
Skip
n_lanes:1 power:100 limit_trust:0.85 control_trust:0.3
Skip
n_lanes:1

In [ ]:
powers = [100, 200, 300, 450, 600, 750, 900, 1050, 1200, 1350,1500,1800,2100,2400,2700,3000,3300]
n_lanes = [11]
control_trusts = [0.1,0.3,0.4, 0.5,0.75,1.0]
limit_trusts = [0.75,0.8,0.85,0.9,0.95,1.0]

for n_lanes in [11]:
    for power in powers:
        #Pass 1
        config = configs.config.modelV1(power,n_lanes=n_lanes)
        grid = grid_gen.Grid(config, configs.folder_config.TRACK_FOLDER + "\\TrackData.csv")
        df = grid.data_generation(verbose=False)
        costs_df = force_frame.Forces.add_Forces_and_constraints(config, df)            
        traj,V = path_opt.Optimisation.run_stack(config, costs_df, show_plots=False, verbose=False, save_run=False)
        df_with_trajectory = traj.select(["tr_x", "tr_y"]).with_columns(
                    [pl.col("tr_x").alias("x_lane0"), pl.col("tr_y").alias("y_lane0")])
        for limit_trust in [0.8,0.9,1.0]:
            for control_trust in [0.1,0.5,1.0]:
                print(f"n_lanes:{n_lanes} power:{power} limit_trust:{limit_trust} control_trust:{control_trust}")
                #Pass 2
                config_pass2 = configs.config.modelV1(power,n_lanes=1)
                config_pass2.pc.control_trust = control_trust
                config_pass2.pc.limit_trust = limit_trust
                grid = grid_gen.Grid(config_pass2, configs.folder_config.TRACK_FOLDER + "\\TrackData.csv")
                df = grid.data_generation_from_existing_track(df_with_trajectory)
                costs_df = force_frame.Forces.add_Forces_and_constraints(config_pass2, df)
                traj_2,V_2 = path_opt.Optimisation.run_stack(config_pass2, costs_df, save_run=True, show_plots=False, verbose=False, suffix="TwoPass")
                


In [ ]:
config.pc.limit_trust = 0.8
costs_df = force_frame.Forces.add_Forces_and_constraints(config, df)

In [ ]:
config.pc.control_trust = 1.0
traj,V = path_opt.Optimisation.run_stack(config, costs_df)